# Polygons
---

Unstructured Grids are often used in the Dynamical Cores of climate models because of their ability to represent complex geometries effectively. Because of the complexity of the geometry, it is often difficult to produce quick and fast visualizations.

As was discussed in the DATA MAPPING section, data variables are typically mapped to the geometric elements that make up the entire unstructured grid (nodes, edges, and faces).

When data is mapped to the faces, each face is shaded with the value of the data variable. We can treat these faces as Polygons on a plane for visualization.


This notebook showcases how UXarray can visualization data as polygons.

## Setup

In [ ]:
import uxarray as ux

file_dir = "../../meshfiles/"

### Grid with Face-Centered Data Variable


In [ ]:
grid_filename_mpas = file_dir + "oQU480.grid.nc"
data_filename_mpas = file_dir + "oQU480.data.nc"
uxds_mpas = ux.open_dataset(grid_filename_mpas, data_filename_mpas)

### Grid with Node-Centered Data Variable

In [ ]:
grid_filename_geoflow = file_dir + "geoflow.grid.nc"
data_filename_geoflow = file_dir + "geoflow.data.nc"
uxds_geoflow = ux.open_dataset(grid_filename_geoflow, data_filename_geoflow)

## Vector Polygon Plots

In [ ]:
uxds_mpas["bottomDepth"].plot.polygons(width=600, height=300)

In [ ]:
Íuxds_mpas["bottomDepth"].plot.polygons(width=600, height=300, xlim=(-20, 0), ylim=(-5, 5))

## Rasterized Polygon Plots

In [ ]:
uxds_mpas["bottomDepth"].plot.polygons(backend='matplotlib')

In [ ]:
uxds_mpas["bottomDepth"].plot.rasterize(method='polygon', width=600, height=300)

In [ ]:
uxds_mpas["bottomDepth"].plot.rasterize(method='polygon', width=600, height=300, xlim=(-20, 0), ylim=(-5, 5))

In [ ]:
(uxds_mpas["bottomDepth"].plot.rasterize(aspect=2, backend='matplotlib', xlim=(-20, 0), ylim=(-5, 5), pixel_ratio=2.0) +
uxds_mpas["bottomDepth"].plot.rasterize(aspect=2, backend='matplotlib', xlim=(-20, 0), ylim=(-5, 5), pixel_ratio=4.0)).opts(fig_size=150)

## Polygons Around Antimeridian

When attempting to visualize unstructured meshes that reside on a sphere, it's necessary to consider the behavior of geometric elements near the Antimeridian. Elements that exist on or cross the antimeridian need to be corrected to properly visualize them. UXarray uses the antimeridian package to split faces along the antimeridian. More details can be found in their [documentation](https://antimeridian.readthedocs.io/en/stable/).


![antimeridian example](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/Earth_map_with_180th_meridian.jpg/640px-Earth_map_with_180th_meridian.jpg)


In [ ]:
(uxds_mpas["bottomDepth"].plot.polygons(aspect=2, backend='matplotlib', xlim=(-185, -175), ylim=(-5, 5), exclude_antimeridian=False) +
uxds_mpas["bottomDepth"].plot.polygons(aspect=2, backend='matplotlib', xlim=(175, 185), ylim=(-5, 5), exclude_antimeridian=False)).opts(fig_size=150)

In [ ]:
(uxds_mpas["bottomDepth"].plot.polygons(aspect=2, backend='matplotlib', xlim=(-185, -175), ylim=(-5, 5), exclude_antimeridian=True) +
uxds_mpas["bottomDepth"].plot.polygons(aspect=2, backend='matplotlib', xlim=(175, 185), ylim=(-5, 5), exclude_antimeridian=True)).opts(fig_size=150)

## Conversion Methods


UXarray represents Unstructured Grids through a set of coordinate and connectivity variables (i.e. `node_lon`, `node_lat`, `face_node_connectivity`, etc.). These variables can be manipulated to generate geometries (in our case Polygons) that can be fed into visualization libraries (i.e. HoloViews, Datashader, Matplotlib, etc.)


### Representation as a `spatialpandas.GeoDataFrame`

The primary conversion


Each face is converted into a Polygon and stored within a SpatialPandas `GeoDataFrame`, which is the expected data structured by the HoloViz stack of packages for visualizing polygons.



A `Grid` can be converted into a `GeoDataFrame`, containing a single "geometry" column, which is a series of Polygons that represent each face.

In [ ]:
uxds_mpas.uxgrid.to_geodataframe()

A `UxDataArray` can also be converted into a `GeoDataFrame`. It will now have an additional column containing a 1D-slice of data variable.

It's important to note that to convert a `UxDataArray` into a `GeoDataFrame`, the dimension of the data variable must match the number of faces (a.k.a. mapped to faces) and there can not be any additional dimensions (i.e. time, level, etc.)

In [ ]:
uxds_mpas["bottomDepth"].to_geodataframe()

If a data variable is not face-centered, it can be manipulated to become face-centered. For node-centered data, as is the case with our Geoflow dataset, we can perform a nodal-average operation, which takes the average all the nodes that surround a face and use that value to shade the polygon.

Here we can also see that we need to index the `time` and `meshLayers` dimensions to obtain our 1D slice of data.

In [ ]:
uxds_geoflow["v1"].nodal_average()[0][0].to_geodataframe()

In [ ]:
uxds_mpas["bottomDepth"].plot.polygons(width=900, height=400)

In [ ]:
uxds_geoflow["v1"].nodal_average()[0][0].plot.polygons(
    cmap="coolwarm", width=900, height=400
)

:::{attention}

:::

<div class="admonition alert alert-warning">
    <p class="admonition-title" style="font-weight:bold">Attention!</p>
    For moderately to high resolution grids, it's not recommended to directly plot polygons. <br>
    Plotting each polygon is extremely computationally expensive to render and may not provide the most visually-appealing plots.
</div>

## Rasterized Polygon Plots

Instead of plotting the geometry of each polygon directly, we can rasterize our set of polygons to obtain a raster plot.

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Hint</p>
    A raster plot of any set of geometric elements (in this case Polygons) renders each data into a regularly shaped array as opposed to rendering each
    shape directly.
</div>

In [ ]:
uxds_mpas["bottomDepth"].plot.rasterize(method="polygon", width=900, height=400)

In [ ]:
uxds_geoflow["v1"].nodal_average()[0][0].plot.rasterize(
    method="polygon", cmap="coolwarm", width=900, height=400
)

You can select whether to include or exclude these antimeridian polygons by using the `exclude_antimeridian` parameter. 

<div class="admonition alert alert-warning">
    <p class="admonition-title" style="font-weight:bold">Attention!</p>
    For larger, higher-resolution, grids, it's suggested to keep `exclude_antimeridian=True` to decrease the time needed to process the grid for
    visualization.
</div>